## 多线程(时间片轮转)

In [7]:
import threading
import time

def dance():
    for i in range(5):
        print("dancing")
        time.sleep(1)

def sing():
    for i in range(5):
        print("singing")
        time.sleep(1)
    
def main():
    dance()
    sing()
    
if __name__ == "__main__":
    main()

dancing
dancing
dancing
dancing
dancing
singing
singing
singing
singing
singing


In [13]:
# 并发随机分配执行
import threading
import time

def dance():
    for i in range(5):
        print("dancing")
        time.sleep(1)

def sing():
    for i in range(5):
        print("singing")
        time.sleep(1)
    
def main():
    t1 = threading.Thread(target=dance, args=())
    t2 = threading.Thread(target=sing, args=())
    t1.start()
    t2.start()
    
if __name__ == "__main__":
    main()

dancing
singing
dancingsinging

singingdancing

dancingsinging

dancing
singing


```
并行：真的多任务
并发：假的多任务
```

In [15]:
a = ["aa", "bb", "cc"]
for num, value in enumerate(a):
    print(num, value)

0 aa
1 bb
2 cc


In [23]:
# 查看当前运行线程
# threading.enumerate()函数查看当前运行线程，返回列表
x = threading.enumerate()
print(x)
print("-"*30)
print("当前运行线程数：%d" % len(x))

[<_MainThread(MainThread, started 4320768832)>, <Thread(Thread-2, started daemon 123145477660672)>, <Heartbeat(Thread-3, started daemon 123145482915840)>, <HistorySavingThread(IPythonHistorySavingThread, started 123145489244160)>, <ParentPollerUnix(Thread-1, started daemon 123145494499328)>]
------------------------------
当前运行线程数：5
